<a href="https://colab.research.google.com/github/gabrielwithappy/Python_study/blob/main/huggingface_chap1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import subprocess
import sys
import os
from pathlib import Path

ROOT = Path('/content/drive/MyDrive/20.Projects/huggingface/')

## Install Libraries

In [ ]:
%cd '/content/drive/MyDrive/20.Projects/huggingface/'
cmd = ['python', '-m', 'pip', 'install']
libs = ["transformers", "datasets", "accelerate", "sentencepiece", "sacremoses"]

cmd += libs
process_install = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print(f'execute {cmd}')
if process_install.stdout:
  print(process_install.stdout)
else:
  print(process_install.stderr)

/content/drive/MyDrive/20.Projects/huggingface
execute ['python', '-m', 'pip', 'install', 'transformers', 'datasets']
b'Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/\nRequirement already satisfied: transformers in /usr/local/lib/python3.8/dist-packages (4.25.1)\nRequirement already satisfied: datasets in /usr/local/lib/python3.8/dist-packages (2.8.0)\nRequirement already satisfied: requests in /usr/local/lib/python3.8/dist-packages (from transformers) (2.25.1)\nRequirement already satisfied: packaging>=20.0 in /usr/local/lib/python3.8/dist-packages (from transformers) (21.3)\nRequirement already satisfied: tqdm>=4.27 in /usr/local/lib/python3.8/dist-packages (from transformers) (4.64.1)\nRequirement already satisfied: regex!=2019.12.17 in /usr/local/lib/python3.8/dist-packages (from transformers) (2022.6.2)\nRequirement already satisfied: huggingface-hub<1.0,>=0.10.0 in /usr/local/lib/python3.8/dist-packages (from transformers) (0.11

## Import Libraries

In [ ]:
import importlib
import torch

is_gpu_available = torch.cuda.is_available()

if is_gpu_available is False:
  print(f'== [Warning] Currently torch is not available ==')

for l in libs:
  m = importlib.import_module(l)
  version = ''
  if hasattr(m, '__version__'):
    print(f"using {m.__name__} {version}")


== [Warning] Currently torch is not available ==
using transformers 
using datasets 
using accelerate 
using sentencepiece 
using sacremoses 


## Huggingface Transformer

### 텍스트 분류

In [ ]:
eng_neg = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

eng_pos = """In his book Happiness: A Very Short Introduction, the philosopher Daniel Haybron supports Meng’s case by citing other researchers, \
            particularly psychologist Michael Argyle who has suggested “only dancing generated higher ‘levels of joy’ than volunteer and charity work.”\
            Fredrickson, too, has studied the benefits of a form of meditation that involved thinking positive thoughts about others. \
            She asked people to try the technique for a few minutes a day for several weeks, and many reported feeling more joyful and hopeful.”"""

korean_neg = """선생님은 공평하지 않다. 오늘 자습시간에 교실에서 떠든 아이는 수길이였는데 선생님은 수길이는 그냥 두고 떠들지도 않은 나를 꾸중하셨다.\
           나는 선생님이 들어오실 때 수길이가 너무 떠들어서 좀 조용히 하라고 했을 뿐인데 나만 꾸중하시니 나를 미워하시는 것 같다. \
           너무 억울하다. 수길이는 나쁜 놈이다. 내가 꾸중을 들을 때 자기는 아무잘못이 없는 것처럼 시치미를 떼고 있었다. 나 같으면 그러지 않았겠다."""

korean_to_eng_neg = """Teachers are not fair. During self-study today, Su-gil was the talkative child in the classroom, but the teacher scolded me for not talking about Su-gil. When the teacher came in, Su-gil was talking too much, so I just told him to be quiet, but he seems to hate me because he only scolded me. It's too unfair. Su-gil is a bad guy. When I was scolded, he acted as if he had done nothing wrong. If it were me, I wouldn't have done that."""

korean_pos = """안녕하세요! 직접 말하거나 전화통하로 하기에는 쑥쓰럽고 망설여지는데 귀여운 이모티콘을 같이 넣어 카톡이나\
           문자메세지로 보내면 한결 수월하겠죠?! 친구나 가족 지인 고객분들에게 문자나 카카오톡으로 보내기 좋은 아침인사말 문구들을 몇개 모아 정리 해보았어요.\
           월요일 아침이나 평일 아침에 누군가로 부터 기분 좋은 인사말, 기분 좋은문자를 받는다면 하루종일 아니 한주를 행복하게 보낼 수 있겠죠?!."""

korean_to_eng_pos = """hello! It's embarrassing and hesitant to talk in person or over the phone, but it would be much easier if you put a cute emoticon and send it via KakaoTalk or text message?! I've collected and organized some good morning greeting phrases to send to friends, family, acquaintances, and customers via text message or KakaoTalk. If you receive a pleasant greeting or a pleasant text message from someone on Monday morning or weekday morning, you will be able to spend the whole week happily, right?!"""

table = {'eng_pos' : eng_pos, 
         'eng_neg' : eng_neg, 
         'korean_pos': korean_pos, 
         'korean_to_eng_pos' : korean_to_eng_pos,
         'korean_neg' : korean_neg,
         'korean_to_eng_neg' : korean_to_eng_neg}

In [ ]:
from transformers import pipeline
import pandas as pd

classifier = pipeline("text-classification")

for k,v in table.items():
  out = classifier(v)
  print(f'{k}')
  display(pd.DataFrame(out))
  print("")
  print("")
  

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


eng_pos


,label,score
0,POSITIVE,0.998279




eng_neg


,label,score
0,NEGATIVE,0.901546


...

,label,score
0,POSITIVE,0.998747




korean_neg


,label,score
0,POSITIVE,0.745737




korean_to_eng_neg


,label,score
0,NEGATIVE,0.997938


## 개체명 인식

In [ ]:
ner_tagger = pipeline("ner", aggregation_strategy="simple")

for k,v in table.items():
  out = ner_tagger(v)
  print(f'{k}')
  display(pd.DataFrame(out))
  print("")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


eng_pos


,entity_group,score,word,start,end
0,PER,0.996333,Daniel Haybron,66,80
1,PER,0.974122,Meng,90,94
2,PER,0.999419,Michael Argyle,169,183
3,PER,0.994210,” Fredrickson,281,305



eng_neg


,entity_group,score,word,start,end
0,ORG,0.879011,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556571,Mega,208,212
4,PER,0.590256,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498349,##icons,259,264
7,MISC,0.775362,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


...

,entity_group,score,word,start,end
0,MISC,0.680980,KakaoT,146,152
1,MISC,0.798771,Kaka,315,319



korean_neg


""



korean_to_eng_neg


,entity_group,score,word,start,end
0,PER,0.999064,Su,48,50
1,PER,0.957573,gil,51,54
2,PER,0.999047,Su,146,148
3,PER,0.928513,gil,149,152
4,PER,0.999070,Su,180,182
5,PER,0.941120,gil,183,186
6,PER,0.999304,Su,310,312
7,PER,0.954169,gil,313,316


## 질문/답변

In [ ]:
reader = pipeline("question-answering")
questions = ["What is the joy?", 
             "What does the customer want?", 
             "카카오톡을 어떻게 사용할까?",
             "How do we use kakao-talk?",
              "수길이의 감정은 무엇일까?", 
             "How about  Su-gil's feeling?"]

for i, (k, v) in enumerate(table.items()):
  outs = reader(question=questions[i], context=v)
  print(f'{k}')
  display(pd.DataFrame([outs]))
  print("")


No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


eng_pos


,score,start,end,answer
0,0.307694,208,215,dancing



eng_neg


,score,start,end,answer
0,0.631292,335,358,an exchange of Megatron


...

,score,start,end,answer
0,0.149938,159,171,text message



korean_neg


,score,start,end,answer
0,0.001905,14,74,오늘 자습시간에 교실에서 떠든 아이는 수길이였는데 선생님은 수길이는 그냥 두고 떠들...



korean_to_eng_neg


,score,start,end,answer
0,0.13859,293,308,It's too unfair


## 요약

In [ ]:
summarizer = pipeline("summarization")

for k,v in table.items():
  out = summarizer(v, max_length=120, clean_up_tokenization_spaces=True)
  print(f'{k}')
  display(pd.DataFrame(out))
  print("")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


eng_pos


,summary_text
0,Researchers have studied the benefits of a fo...


Your max_length is set to 120, but you input_length is only 111. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=55)



eng_neg


...


korean_neg


,summary_text
0,선생님� 공평하지 - ‘“ ”� “”” ‘I’ll be abl...


Your max_length is set to 120, but you input_length is only 116. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=58)



korean_to_eng_neg


,summary_text
0,Su-gil was the talkative child in the classro...


## 번역

In [ ]:
translator = pipeline("translation_en_to_de", 
                      model="Helsinki-NLP/opus-mt-ko-en")
outputs = translator(table['korean_pos'], clean_up_tokenization_spaces=True, min_length=100)
print("Hugging Face")
print(outputs[0]['translation_text'])
print("안녕하세요 전화로 말하기 쑥스럽거나 망설이시는 분들도 카톡이나 문자에 귀여운 이모티콘을 넣어주시면 친구들의 좋은 아침인사를 받을 수 있어 행복할 것 같아요 또는 가족 손님, 그리고 월요일 아침이나 그 날 아침에 누군가로부터 반가운 인사를 보내면 기뻐할 것입니다. 당신에게 말하십시오.")
print("")
print("")
print("Google")
print(table['korean_to_eng_pos'])
print("안녕하세요! 직접 만나거나 전화로 이야기하는 것도 쑥스럽고 주저하지만 귀여운 이모티콘을 넣어 카톡이나 문자로 보내주시면 훨씬 수월하겠죠?! 친구, 가족, 지인, 고객에게 문자나 카카오톡으로 보낼 좋은 아침 인사 문구를 모아 정리했습니다. 월요일 아침이나 평일 오전에 누군가로부터 반가운 인사나 기분 좋은 문자를 받는다면 일주일 내내 행복하게 보낼 수 있겠죠?!")
# https://huggingface.co/Helsinki-NLP/opus-mt-ko-en

Hugging Face
Hi, you're embarrassed to say it or you're hesitant to do it on the phone, but if you put a cute emoji in a katok or a text message, you'd be happy to get some good morning greetings for your friends or family guests, and you'd be happy to send them a nice greeting from someone on Monday morning, or on the morning of the day, if they're happy to say it to you all day, or if they're happy to say it to you.
안녕하세요 전화로 말하기 쑥스럽거나 망설이시는 분들도 카톡이나 문자에 귀여운 이모티콘을 넣어주시면 친구들의 좋은 아침인사를 받을 수 있어 행복할 것 같아요 또는 가족 손님, 그리고 월요일 아침이나 그 날 아침에 누군가로부터 반가운 인사를 보내면 기뻐할 것입니다. 당신에게 말하십시오.


Google
hello! It's embarrassing and hesitant to talk in person or over the phone, but it would be much easier if you put a cute emoticon and send it via KakaoTalk or text message?! I've collected and organized some good morning greeting phrases to send to friends, family, acquaintances, and customers via text message or KakaoTalk. If you receive a pleasant greeting or a pleasant text message from someone on Monday

## 택스트 생성

In [ ]:
from transformers import set_seed
set_seed(42) # 동일 결과를 재현하기 위해 지정

### 정보 미제공

In [ ]:
generator = pipeline("text-generation")
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. I have received the order (I'm sure your husband would love to have a rematch!) and it is not returning my order, I would like to confirm that the item was not delivered. I will be purchasing the rematch and can not give you the correct message.


Thanks again for your honest effort!


PLEASE ACCEPT THIS INFORMATION WITH A RESPONSE TO YOUR FOLLOWING ME - Please do not repeat the mistake you made here. Please, do not send me an email, message, or any other message you do not understand or can't understand. Bumblebee,

Betsy


Customer service response:

Hi, I'm sorry, I'm sorry. I received your order. Please follow up with me at your message office. Thank you.


Thank you,

Betsy


Customer service response:


### 정보 제공

In [ ]:
generator = pipeline("text-generation")
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = table['eng_neg'] + "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. Enclosed are copies of my records concerning this purchase. I expect to hear from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. As you are aware, the Megatron toy was purchased in Germany from a good-good wholesaler. I could not confirm the situation from my original email address, but I am fairly sure someone from your warehouse sent it to me. This is a very small issue where you may have an issue with shipping charges too. However, I


In [ ]:
# 1. Models > tasks::'translation' > search "ko"
# translator = pipeline("translation_en_to_de", 
#                       model="Helsinki-NLP/opus-mt-ko-en")
# https://github.com/huggingface/transformers/blob/main/src/transformers/pipelines/text2text_generation.py#L336

SyntaxError: ignored